In [1]:
import sys
import pandas as pd
import pymongo
import json
import os
import pprint
from pandas.io.json import json_normalize
from sqlalchemy import create_engine

In [2]:
file = '4.1.gtd.csv'
terror_df = pd.read_csv(file, encoding = "ISO-8859-1")

terror_df.head()

,eventid,iyear,country_txt,region,region_txt,provstate,latitude,longitude,success,natlty1,natlty1_txt,gname,individual,nperps,nperpcap,nkill,nwound
0,1.997010e+11,1997,Nepal,6,South Asia,Central,27.493003,85.824033,1,141.0,Nepal,Unknown,0,300.0,NaN,5.0,2.0
1,1.997010e+11,1997,West Bank and Gaza Strip,10,Middle East & North Africa,West Bank,31.530243,35.094162,1,155.0,West Bank and Gaza Strip,Jewish Extremists,0,1.0,NaN,0.0,7.0
2,1.997010e+11,1997,United States,1,North America,Oklahoma,36.149745,-95.993334,1,217.0,United States,Anti-Abortion extremists,1,NaN,1.0,0.0,0.0
3,1.997010e+11,1997,United States,1,North America,Oklahoma,36.149745,-95.993334,1,217.0,United States,Anti-Abortion extremists,1,NaN,1.0,0.0,0.0
4,1.997010e+11,1997,United States,1,North America,Oklahoma,36.149745,-95.993334,1,217.0,United States,Anti-Abortion extremists,1,NaN,1.0,0.0,0.0


In [4]:
heat_df = terror_df[["latitude","longitude","eventid"]]
heat_df.head()

,latitude,longitude,eventid
0,27.493003,85.824033,1.997010e+11
1,31.530243,35.094162,1.997010e+11
2,36.149745,-95.993334,1.997010e+11
3,36.149745,-95.993334,1.997010e+11
4,36.149745,-95.993334,1.997010e+11


In [7]:
heat_df.set_index('eventid',inplace=True)
heat_df.head()

,latitude,longitude
eventid,,
1.997010e+11,27.493003,85.824033
1.997010e+11,31.530243,35.094162
1.997010e+11,36.149745,-95.993334
1.997010e+11,36.149745,-95.993334
1.997010e+11,36.149745,-95.993334


In [20]:
zoom_df = terror_df[["region","country_txt","provstate"]]
zoom_df.set_index('country_txt',inplace=True)
zoom_df.head()

,region,provstate
country_txt,,
Nepal,6,Central
West Bank and Gaza Strip,10,West Bank
United States,1,Oklahoma
United States,1,Oklahoma
United States,1,Oklahoma


In [28]:
bar_df=terror_df.groupby(['natlty1_txt', 'gname', ]).size().unstack(level=1, fill_value=0)
bar_df.head()

gname,Al-Aqsa Martyrs Brigade,Al-Fatah,Al-Qaida,Al-Qaida in the Islamic Maghreb (AQIM),Crypteia,Gholam Yahya Akbar,Gunmen,Haqqani Network,Hizb-I-Islami,Insurgents,...,Southern Mobility Movement (Yemen),Southern Yemen Separatists,Supporters of Abd Rabbuh Mansur Hadi,Supporters of Saddam Hussein,Tehama Movement,United Party for National Development (UPND),Liberation War Veterans Association,Movement for Democratic Change (MDC),National Youth Service of Zimbabwe,Zimbabwe African Nationalist Union (ZANU)
natlty1_txt,,,,,,,,,,,,,,,,,,,,,
Afghanistan,3,1,17,2,1,1,8,49,24,1,...,0,0,0,0,0,0,0,0,0,0
Albania,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Algeria,0,0,0,172,0,0,22,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Angola,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Argentina,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
line_df=terror_df.groupby(['iyear', 'country_txt']).size().unstack(level=1, fill_value=0)
line_df.head()

country_txt,Afghanistan,Albania,Algeria,Angola,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,...,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza Strip,Western Sahara,Yemen,Yugoslavia,Zaire,Zambia,Zimbabwe
iyear,,,,,,,,,,,,,,,,,,,,,
1997,1,41,344,7,11,0,4,1,1,2,...,0,41,2,22,0,19,38,5,0,0
1998,1,7,151,20,0,2,6,1,0,0,...,0,1,0,12,0,7,46,0,0,0
1999,9,3,106,34,1,2,0,0,0,0,...,8,5,0,11,0,17,23,0,7,0
2000,14,2,138,22,0,2,1,0,1,0,...,0,0,0,28,0,10,14,0,8,1
2001,14,1,113,40,2,2,2,0,4,0,...,0,5,4,123,0,7,20,0,0,3
